# 2023-07-10__AdEx_Nto1_Brian_speedtest

In [1]:
%run lib/neuron.py

importing numpy, brian … ✔


We're gonna try a brian trick to try and speed up the simulation:
instead of 6500 separate Poisson spike trains, simulate most of them (except a few that we want to use for conntesting) as **one** process. (One for exc and one for inh, to be precise).

In [2]:
μₓ = 4 * Hz
σ = sqrt(0.6)
μ = log(μₓ / Hz) - σ**2 / 2

1.0862943611198905

## `PoissonGroup` only

In [3]:
def Nto1_all_simmed(N = 6500):
    
    Ne = N * 4//5
    
    n = COBA_AdEx_neuron()
    
    rates = lognormal(μ, σ, N) * Hz
    P = PoissonGroup(N, rates)
    
    Se = Synapses(P, n, on_pre="ge += we")
    Si = Synapses(P, n, on_pre="gi += wi")
    Se.connect("i < Ne")
    Si.connect("i >= Ne")
    
    M = StateMonitor(n, ["V"], record=[0])
    S = SpikeMonitor(n)
    SP = SpikeMonitor(P)
    
    objs = [n, P, Se, Si, M, S, SP]
    return *objs, Network(objs)

In [4]:
%%time
*objs, net = Nto1_all_simmed();
net.store()

CPU times: total: 438 ms
Wall time: 776 ms


In [5]:
we = 14 * pS
wi = 4 * we

T = 10*second;

In [6]:
%%time
net.restore()
net.run(T, report='text')

Starting simulation at t=0. s for a duration of 10. s
1.7002 s (17%) simulated in 10s, estimated 49s remaining.
3.3176 s (33%) simulated in 20s, estimated 40s remaining.
5.1558 s (51%) simulated in 30s, estimated 28s remaining.
6.8499 s (68%) simulated in 40s, estimated 18s remaining.
8.5327 s (85%) simulated in 50s, estimated 9s remaining.
10. s (100%) simulated in 58s
CPU times: total: 27.9 s
Wall time: 59.2 s


So, 19 s for the sim itself,  
29 s for the whole thing, i.e. setup took 10 seconds (initial compilation, ig).

..that compil gets cached though. Second time: no init time at all.
(Neither at `run` time or at the objs creation time).

2nd try, other day: 51 s for whole sim.  
(total of block: 53.5 s).

3rd try, this same day: 58 s for whole sim.

(weird, nothin shoulda changed, why 2+ x slower)

## `PoissonGroup` + `PoissonInput`s (merged)

In [7]:
def Nto1_merged(N = 6500, Ne_simmed = 100):
    
    Ni_simmed = Ne_simmed

    Ne = N * 4//5
    Ni = N - Ne
    Ne_merged = Ne - Ne_simmed
    Ni_merged = Ni - Ni_simmed
    N_simmed = Ne_simmed + Ni_simmed
    print(f"{Ne=}, {Ni=}, {N_simmed=}, {Ne_merged=}, {Ni_merged=}")
    
    n = COBA_AdEx_neuron()

    rates = lognormal(μ, σ, N_simmed) * Hz
    P = PoissonGroup(N_simmed, rates)
    Se = Synapses(P, n, on_pre="ge += we")
    Si = Synapses(P, n, on_pre="gi += wi")
    Se.connect("i < Ne_simmed")
    Si.connect("i >= Ne_simmed")

    PIe = PoissonInput(n, 'ge', Ne_merged, μₓ, we)
    PIi = PoissonInput(n, 'gi', Ni_merged, μₓ, wi);

    M = StateMonitor(n, ["V"], record=[0])
    S = SpikeMonitor(n)
    SP = SpikeMonitor(P)
    
    objs = [n, P, Se, Si, PIe, PIi, M, S, SP]
    return *objs, Network(objs)

In [8]:
%%time
*objs_m, net_m = Nto1_merged()
net_m.store()

Ne=5200, Ni=1300, N_simmed=200, Ne_merged=5100, Ni_merged=1200
CPU times: total: 219 ms
Wall time: 312 ms


In [9]:
%%time
net_m.restore()
net_m.run(T, report='text')

Starting simulation at t=0. s for a duration of 10. s
3.3382 s (33%) simulated in 10s, estimated 20s remaining.
7.3675 s (73%) simulated in 20s, estimated 7s remaining.
10. s (100%) simulated in 27s
CPU times: total: 13.8 s
Wall time: 27.7 s


29 s for 10".

whole block: 64 sec.

And again, this same other day: 27 s.

So, speedup of that PInput:

In [11]:
(51+58)/2

54.5

In [10]:
(51+58) / (27 + 29)

1.9464285714285714

2x. worth the extra complexity? eh, sure.

Otoh, what do when less than 6500 inputs.
Say, 100 inh, 400 exc.
Or, 10 inh, 40 exc.
(ok, sol is simple, you take max)

## 

How 'bout Julia?

See notebook `2023-08-02__Julia_speedtest_AdEx_Nto1`.\
Simulating all 6500 inputs for 10 seconds, we have a sim time of 0.7 seconds.  
(10 minutes takes 9 seconds).  
I.e. 25x as fast as Brian2 w/ Cython (18 sec).

What about Brian's standalone mode? i.e. no python for main loop; generate a C++ proj.

(does that work with changing params in a loop? (i.e. seed, we))

See `2023-08-02__speedtest_brian_standalone_AdEx_Nto1`.  \
Also < 1 sec.

With `time ./main.exe`: ~0.5 seconds.

Note that that is the one w/ merged inputs.

Without: 8 seconds. Interesting.